In [ ]:
import os  
import base64
from openai import AzureOpenAI  
from langchain_openai import ChatOpenAI, AzureChatOpenAI


endpoint = os.getenv("", "")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version = "2024-12-01-preview"
)

    
# Generate the completion  
def chat_completion_4o(USER_MSG):
    completion = client.chat.completions.create(  
        model=deployment,
        messages=[
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are going to identify the corresponding sense tag of an ambiguous word."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": USER_MSG
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": ""
            }
        ]
    }
] ,
        max_tokens=500,  
        temperature=0.0,
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
        )
    response_content = completion.choices[0].message.content
    return response_content

In [ ]:
import os
from openai import AzureOpenAI

endpoint = ""
model_name = "gpt-4o"
deployment = "gpt-4o"

subscription_key = ""
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

# Generate the completion  
def chat_completion_4o(USER_MSG):
    completion = client.chat.completions.create(  
        model=deployment,
        messages=[
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are going to identify the corresponding sense tag of an ambiguous word."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": USER_MSG
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": ""
            }
        ]
    }
] ,
        max_tokens=500,  
        temperature=0.0,
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
        )
    response_content = completion.choices[0].message.content
    return response_content

In [ ]:
!pip install together

In [ ]:
import os
from together import Together

client = Together() # auth defaults to os.environ.get("TOGETHER_API_KEY")

def chat_completion_deepseek(user_msg: str) -> str:   

  response = client.chat.completions.create(
      model="deepseek-ai/DeepSeek-V3",
      messages=[
        {
          "role": "user",
          "content": user_msg
        }
      ]
  )
  return(response.choices[0].message.content)

In [ ]:
! pip install -U langchain langchain-core



In [ ]:
from huggingface_hub import login
login("")

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain.chains import RetrievalQA


from langchain_openai import ChatOpenAI

In [ ]:
# Files and labels
file_paths = {
    "ambiguous": "human_ambigous.txt",
    "easy": "human_easy.txt",
    "unlikely": "human_unlikely.txt"
}

In [ ]:
def load_and_split(file_path):
    """Load and split a text file using \n as separator."""
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n"],
        chunk_size=500,
        chunk_overlap=50
    )
    split_docs = splitter.split_documents(documents)
    return split_docs

In [ ]:
def load_vector_store(store_name):
    """Load an existing FAISS vector store from disk."""
    embedding_model = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en-v1.5",
        model_kwargs={'device': 'cpu'},  # change to 'cuda' if you have GPU
        encode_kwargs={'normalize_embeddings': True}
    )
    
    return FAISS.load_local(
        f"vectorstore_{store_name}",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # required for FAISS in some LangChain versions
    )

# -------------------------------
# Step 3: Load all your stores
# -------------------------------
vector_stores = {
    "ambiguous": load_vector_store("ambiguous"),
    "easy": load_vector_store("easy"),
    "unlikely": load_vector_store("unlikely")
}

In [ ]:
def retrieve_from_all_stores(query, k=1):
    """Retrieve top-k results from each vector store."""
    results = {}
    for label, store in vector_stores.items():
        docs = store.similarity_search(query, k=k)
        results[label] = docs
    return results

In [ ]:
def rag_pipeline(query, k=3):
    """
    Retrieve top-k documents from each vector store
    and return them as combined and structured strings.
    """
    retrieved = retrieve_from_all_stores(query, k=k)

    retrieved_texts = {}
    for label, docs in retrieved.items():
        if docs:
            # Join all retrieved chunks for that store
            combined_chunks = "\n---\n".join([d.page_content.strip() for d in docs])
            retrieved_texts[label] = combined_chunks
        else:
            retrieved_texts[label] = ""

    # Combine everything into one formatted string
    combined_text = "\n\n".join(
        [f"### {label.upper()} CONTEXT ###\n{content}"
         for label, content in retrieved_texts.items() if content]
    )

    return combined_text, retrieved_texts

In [ ]:
query = "a pair of parallel rails providing a runway for wheels,The detectives arrived at the abandoned train station. They were looking for signs of the missing artifact. A faint trail caught their attention.,They followed the <WSD>track</WSD>.,They began to run along the abandoned railway line, hopping from wooden sleeper to sleeper to avoid twisting an ankle.How do humans react to ambiguous situations?"
combined_text, retrieved_texts = rag_pipeline(query)

print("🔹 Combined Text:\n")
print(combined_text)

In [ ]:
import csv
def main():
    file_path = ''
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"
            
            pre_context = f''' **Story:**  Precontext: {precontext} - Sentence: {sentence} - Ending: {ending}  Homonym Word: "{homonym}'''
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''Your task is to simulate the judgments of 5 independent human annotators who are rating the plausibility of a proposed meaning of a homonymous word on a 1 to 5 scale, based on a short story.

Each annotator reasons independently but follows the same structured reasoning process described below. Differences among annotator scores should naturally arise from ambiguity, weak evidence, or conflicting cues in the story.

You will be provided with example stories illustrating the scoring rubric at different plausibility levels. In those examples, “Annotations by 5 annotators” demonstrate how agreement or disagreement emerges depending on contextual clarity.
{combined_text}

Now evaluate the following story against the proposed meaning.

**Story:**
* **Precontext:** {precontext}
* **Sentence:** {sentence}
* **Ending:** {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

### Thinking Process & Response Format

*Complete each step of this process in your analysis. Each annotator completes the following steps internally before assigning a score.*

**Step 1: Initial Context Analysis**
* Read the **Precontext** *alone*. Which meaning(s) of "{homonym}" does it make you expect?
* Read the **Ambiguous Sentence** *alone*. Is the homonym part of a common idiom (e.g., "had guts") or a set phrase (e.g., "drawing water")?

**Step 2: The Ending as a Clarifier**
* Now, read the **Ending**. How does this new information *shift* the story?
* Does the Ending strongly *confirm* one meaning, *contradict* another, or introduce a *twist*?
* **Crucially:** Does the Ending act as a *paraphrase* or *definition* of a figurative or idiomatic meaning (e.g., an ending of "She was very brave" is a direct definition for the meaning "fortitude")?

**Step 3: Holistic Coherence Check**
* Read all three parts (Precontext, Sentence, Ending) as a *single, coherent narrative*.
* Can all parts co-exist with the **Proposed Meaning**? Or do some parts create a conflict?
* **Supporting Clues:** List the specific words/phrases from the *entire* story (Precontext, Sentence, or Ending) that support the **Proposed Meaning**.
* **Conflicting Clues:** List the specific words/phrases that contradict or make the **Proposed Meaning** awkward or implausible.

**Step 4: Final Score and Justification**
* Based on your coherence check in Step 3, assign a score.
* *Pay close attention to contradictions*: If a strong clue in the Ending directly conflicts with a meaning , the score must be low, *even if* the precontext supported it.


**Scoring Rubric:**
* **5: Perfectly plausible.** The meaning is strongly supported by the *entire* context, and all parts of the story (Precontext, Sentence, Ending) form a consistent, logical narrative.
* **4: Very plausible.** The meaning fits well and is consistent. There might be minor ambiguity, but no real contradictions.
* **3: Moderately plausible.** The meaning is possible, but the context is ambiguous or contains *minor* conflicting clues that make the story a bit strange.
* **2: Barely plausible.** The meaning *largely* conflicts with the context. You have to ignore a key piece of information (especially in the Ending) to make it fit.
* **1: Implausible.** The meaning is directly and strongly contradicted by the context (e.g., the Ending makes it physically impossible or nonsensical).

Perform all reasoning privately. Simulate the final judgments of 5 annotators, reflecting:

High agreement when the meaning is clear,
Partial disagreement when the story is ambiguous,
Strong disagreement when cues conflict.

Output ONLY five comma-separated integers between 1 and 5 (e.g., 5,5,4,5,4). Do not include explanations, text, or formatting beyond the scores.'''
        
                    
            # Generate response using the same format as training
            #print(prompt)
            full_text = chat_completion_4o(prompt)
            full_text_fin=full_text.replace('\n', ' ')
            print(f"Line {line_num}: {full_text_fin}")
            line_num += 1
            


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = ''
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"
            
            pre_context = f''' **Story:**  Precontext: {precontext} - Sentence: {sentence} - Ending: {ending}  Homonym Word: "{homonym}'''
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''Your task is to simulate the judgments of 5 independent human annotators who are rating the plausibility of a proposed meaning of a homonymous word on a 1 to 5 scale, based on a short story.

Each annotator reasons independently but follows the same structured reasoning process described below. Differences among annotator scores should naturally arise from ambiguity, weak evidence, or conflicting cues in the story.

You will be provided with example stories illustrating the scoring rubric at different plausibility levels. In those examples, “Annotations by 5 annotators” demonstrate how agreement or disagreement emerges depending on contextual clarity. Also at the end, final Plausibility Score is provided which is the most probable output based on the average score of 5 annotators.
{combined_text}

Now simulate the evaluation the following story against the proposed meaning. Follow the given instructions.

**Story:**
* **Precontext:** {precontext}
* **Sentence:** {sentence}
* **Ending:** {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"


Complete each step of this process in your analysis.

1. Read the **Precontext** *alone*. Which meaning(s) of "{homonym}" does it make you expect?
2. Read the **Ambiguous Sentence** *alone*. Decide whether the homonym part is a common idiom or a set phrase.
3. Now, read the **Ending**. How does this new information *shift* the story?
4. Does the Ending strongly *confirm* one meaning, *contradict* another, or introduce a *twist*?
5. Now Read all three parts (Precontext, Sentence, Ending) as a *single, coherent narrative*.
6. Decide whether all parts co-exist with the **Proposed Meaning**? Or do some parts create a conflict?
7. Based on the above steps, Assign a final score.


**Scoring Rubric:**
1: The displayed meaning is not plausible at all given the context.
2: The displayed meaning is theoretically conceivable, but less plausible than other meanings.
3: The displayed meaning represents one of multiple, similarly plausible interpretations.
4: The displayed meaning represents the most plausible interpretation; other meanings may still be conceivable.
5: The displayed meaning is the only plausible meaning given the context.

Perform all reasoning privately. Simulate the final judgments of 5 annotators and decide the final plausibility score.

Output ONLY single integer between 1 and 5 (e.g., 3). Do not include explanations, text, or formatting beyond the scores.'''
        
                    
            # Generate response using the same format as training
            #print(prompt)
            full_text = chat_completion_4o(prompt)
            full_text_fin=full_text.replace('\n', ' ')
            print(f"Line {line_num}: {full_text_fin}")
            line_num += 1
            


if __name__ == "__main__":
    main()

In [ ]:
import csv
def main():
    file_path = ''
    
    with open(file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        line_num = 1
        for row in reader:
            # Assign values from each row to variables
            homonym = row["homonym"]
            judged_meaning = row["judged_meaning"]
            precontext = row["precontext"]
            sentence = row["sentence"]
            ending = row["ending"]
            choice = row["choices"]
            example_sentence = row["example_sentence"]
            average = row["average"]

            if ending.strip() == "":
                ending = "[NO ENDING]"
            
            pre_context = f''' **Story:**  Precontext: {precontext} - Sentence: {sentence} - Ending: {ending}  Homonym Word: "{homonym}'''
            combined_text, retrieved_texts = rag_pipeline(pre_context)

            prompt=f'''Your task is to simulate the judgments of 5 independent human annotators who are rating the plausibility of a proposed meaning of a homonymous word on a 1 to 5 scale, based on a short story.

Each annotator reasons independently but follows the same structured reasoning process described below. Differences among annotator scores should naturally arise from ambiguity, weak evidence, or conflicting cues in the story.

You will be provided with example stories illustrating the scoring rubric at different plausibility levels. In those examples, “Annotations by 5 annotators” demonstrate how agreement or disagreement emerges depending on contextual clarity.
{combined_text}

Now evaluate the following story against the proposed meaning.

**Story:**
* **Precontext:** {precontext}
* **Sentence:** {sentence}
* **Ending:** {ending}

**Word:** "{homonym}"
**Proposed Meaning to Evaluate:** "{judged_meaning}"

### Thinking Process & Response Format

*Complete each step of this process in your analysis. Each annotator completes the following steps internally before assigning a score.*

**Step 1: Initial Context Analysis**
* Read the **Precontext** *alone*. Which meaning(s) of "{homonym}" does it make you expect?
* Read the **Ambiguous Sentence** *alone*. Is the homonym part of a common idiom (e.g., "had guts") or a set phrase (e.g., "drawing water")?

**Step 2: The Ending as a Clarifier**
* Now, read the **Ending**. How does this new information *shift* the story?
* Does the Ending strongly *confirm* one meaning, *contradict* another, or introduce a *twist*?
* **Crucially:** Does the Ending act as a *paraphrase* or *definition* of a figurative or idiomatic meaning (e.g., an ending of "She was very brave" is a direct definition for the meaning "fortitude")?

**Step 3: Holistic Coherence Check**
* Read all three parts (Precontext, Sentence, Ending) as a *single, coherent narrative*.
* Can all parts co-exist with the **Proposed Meaning**? Or do some parts create a conflict?
* **Supporting Clues:** List the specific words/phrases from the *entire* story (Precontext, Sentence, or Ending) that support the **Proposed Meaning**.
* **Conflicting Clues:** List the specific words/phrases that contradict or make the **Proposed Meaning** awkward or implausible.

**Step 4: Final Score and Justification**
* Based on your coherence check in Step 3, assign a score.
* *Pay close attention to contradictions*: If a strong clue in the Ending directly conflicts with a meaning , the score must be low, *even if* the precontext supported it.


**Scoring Rubric:**
* **5: Perfectly plausible.** The meaning is strongly supported by the *entire* context, and all parts of the story (Precontext, Sentence, Ending) form a consistent, logical narrative.
* **4: Very plausible.** The meaning fits well and is consistent. There might be minor ambiguity, but no real contradictions.
* **3: Moderately plausible.** The meaning is possible, but the context is ambiguous or contains *minor* conflicting clues that make the story a bit strange.
* **2: Barely plausible.** The meaning *largely* conflicts with the context. You have to ignore a key piece of information (especially in the Ending) to make it fit.
* **1: Implausible.** The meaning is directly and strongly contradicted by the context (e.g., the Ending makes it physically impossible or nonsensical).

Perform all reasoning privately. Simulate the final judgments of 5 annotators, reflecting:

High agreement when the meaning is clear,
Partial disagreement when the story is ambiguous,
Strong disagreement when cues conflict or story is implausible.

Output ONLY five comma-separated integers between 1 and 5 (e.g., 3,5,4,5,4). Do not include explanations, text, or formatting beyond the scores.'''
        
                    
            # Generate response using the same format as training
            #print(prompt)
            full_text = chat_completion_deepseek(prompt)
            full_text_fin=full_text.replace('\n', ' ')
            print(f"Line {line_num}: {full_text_fin}")
            line_num += 1
            


if __name__ == "__main__":
    main()

In [ ]:
import json

def save_predictions_to_jsonl(numbers, file_path):
    """
    Saves a list of numbers to a JSONL file in the format:
    {"id": "<index>", "prediction": <number>}
    
    Args:
        numbers (list): List of numbers (predictions).
        file_path (str): Path to output .jsonl file.
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for idx, num in enumerate(numbers):
            record = {
                "id": str(idx),
                "prediction": num
            }
            f.write(json.dumps(record) + "\n")

In [ ]:
predicts=[]
from collections import Counter
with open('results_output_GPT.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for item in lines:
        elements = item.split(': ')[1].strip()
        list_numbers = [int(x.strip()) for x in elements.split(',')]
        # predicts.append(sum(list_numbers)/5)
        most_frequent = Counter(list_numbers).most_common(1)[0][0]
        predicts.append(most_frequent)
print(predicts)
print(len(predicts))
save_predictions_to_jsonl(predicts, 'predictions_GPT_most.jsonl')
       

In [ ]:
predictsdeep=[]
from collections import Counter
with open('resultoutput_deepseek.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for item in lines:
        elements = item.split(': ')[1].strip()
        list_numbers = [int(x.strip()) for x in elements.split(',')]
        # predictsdeep.append(sum(list_numbers)/5)
        most_frequent = Counter(list_numbers).most_common(1)[0][0]
        predictsdeep.append(most_frequent)
print(predictsdeep)
print(len(predictsdeep))
# save_predictions_to_jsonl(predictsdeep, 'predictions_deepseek.jsonl')

In [ ]:
predictsfinal=[]
for gpt, deep in zip(predicts, predictsdeep):
    mean = (gpt + deep) / 2
    predictsfinal.append(round(mean))
print(predictsfinal)
save_predictions_to_jsonl(predictsfinal, 'predictions_final.jsonl')

In [ ]:
gpt=[5, 2, 5, 3, 5, 3, 2, 3, 2, 3, 2, 3, 3, 3, 1, 5, 3, 4, 5, 4, 5, 5, 5, 4, 4, 2, 3, 2, 4, 2, 3, 5, 3, 5, 4, 5, 4, 4, 3, 5, 4, 5, 3, 4, 3, 4, 3, 4, 4, 2, 3, 5, 3, 5, 4, 4, 3, 5, 4, 5, 5, 2, 2, 5, 4, 2, 4, 2, 1, 5, 2, 2, 5, 2, 2, 5, 5, 4, 5, 3, 2, 5, 4, 5, 2, 5, 2, 5, 2, 5, 2, 3, 2, 4, 3, 4, 4, 2, 5, 2, 5, 3, 5, 2, 2, 5, 2, 5, 5, 1, 4, 3, 5, 2, 5, 1, 2, 5, 5, 1, 5, 3, 5, 3, 5, 4, 5, 3, 2, 4, 2, 4, 5, 2, 2, 3, 3, 3, 5, 5, 4, 5, 4, 5, 2, 5, 2, 5, 2, 5, 5, 2, 2, 5, 5, 2, 5, 2, 2, 5, 5, 2, 2, 5, 2, 5, 2, 5, 3, 1, 5, 1, 3, 1, 5, 2, 1, 4, 2, 3, 4, 3, 2, 4, 3, 4, 3, 2, 2, 5, 3, 3, 5, 2, 2, 3, 4, 2, 4, 2, 3, 2, 4, 2, 4, 2, 2, 5, 3, 3, 3, 2, 3, 3, 3, 3, 5, 3, 2, 4, 3, 4, 5, 2, 3, 4, 4, 4, 2, 3, 2, 3, 2, 3, 5, 5, 3, 5, 4, 5, 2, 5, 2, 5, 2, 5, 5, 2, 2, 2, 5, 2, 5, 3, 5, 2, 5, 4, 2, 5, 2, 5, 2, 5, 3, 3, 2, 3, 3, 5, 5, 2, 2, 5, 3, 3, 2, 5, 2, 5, 2, 4, 3, 5, 3, 5, 4, 5, 5, 4, 2, 5, 4, 5, 2, 2, 2, 5, 2, 5, 3, 2, 3, 5, 4, 3, 4, 2, 3, 3, 4, 3, 3, 3, 2, 3, 2, 3, 5, 2, 5, 2, 5, 2, 5, 2, 4, 2, 5, 2, 4, 2, 1, 5, 2, 4, 5, 2, 2, 5, 4, 2, 5, 2, 3, 4, 4, 3, 5, 2, 2, 3, 2, 2, 5, 2, 4, 3, 4, 2, 5, 2, 5, 1, 3, 2, 4, 2, 1, 4, 2, 3, 5, 2, 5, 2, 3, 3, 3, 2, 2, 3, 3, 2, 5, 1, 4, 2, 5, 1, 5, 2, 2, 4, 5, 2, 5, 2, 4, 4, 5, 3, 2, 5, 2, 4, 2, 4, 2, 1, 2, 3, 2, 1, 5, 1, 3, 2, 5, 2, 2, 3, 4, 3, 3, 3, 5, 2, 4, 3, 4, 3, 4, 2, 4, 2, 3, 2, 5, 2, 4, 3, 4, 3, 4, 2, 3, 2, 4, 3, 4, 2, 2, 5, 4, 4, 4, 2, 3, 5, 4, 3, 2, 5, 3, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 4, 3, 5, 2, 2, 5, 2, 3, 2, 4, 1, 5, 2, 4, 5, 3, 5, 2, 5, 2, 4, 2, 2, 5, 2, 4, 5, 2, 2, 4, 2, 2, 5, 3, 2, 5, 2, 4, 5, 2, 2, 5, 2, 4, 2, 3, 1, 5, 2, 4, 2, 3, 1, 5, 3, 4, 5, 5, 5, 5, 4, 5, 2, 4, 2, 5, 2, 5, 2, 2, 2, 5, 2, 5, 1, 4, 2, 5, 2, 5, 5, 2, 1, 5, 2, 4, 5, 5, 4, 4, 4, 5, 5, 2, 3, 4, 4, 3, 5, 2, 2, 5, 2, 5, 4, 1, 2, 2, 3, 1, 2, 4, 2, 5, 2, 4, 2, 4, 2, 4, 2, 4, 5, 1, 5, 2, 5, 1, 3, 2, 1, 5, 2, 4, 5, 2, 2, 5, 2, 4, 5, 2, 3, 4, 3, 4, 3, 4, 2, 5, 3, 4, 5, 2, 5, 2, 4, 2, 3, 2, 2, 2, 4, 2, 5, 2, 3, 3, 5, 2, 2, 4, 2, 5, 3, 4, 4, 2, 2, 5, 3, 4, 4, 3, 2, 4, 3, 4, 5, 2, 3, 5, 5, 2, 4, 5, 2, 5, 3, 5, 5, 5, 2, 5, 3, 5, 2, 5, 2, 5, 2, 5, 3, 3, 2, 5, 2, 4, 3, 5, 3, 5, 2, 5, 2, 2, 2, 4, 2, 4, 5, 1, 2, 4, 3, 1, 5, 5, 2, 4, 3, 4, 3, 4, 4, 5, 3, 5, 4, 2, 2, 5, 4, 2, 5, 2, 2, 5, 3, 4, 5, 1, 2, 5, 5, 3, 5, 1, 5, 2, 5, 2, 2, 2, 1, 5, 2, 5, 5, 2, 2, 5, 3, 3, 5, 1, 5, 2, 5, 2, 2, 2, 2, 5, 4, 3, 3, 5, 5, 2, 4, 3, 5, 2, 2, 5, 3, 3, 2, 2, 2, 5, 2, 5, 4, 2, 2, 5, 2, 4, 2, 5, 4, 4, 4, 3, 3, 3, 2, 3, 2, 4, 5, 2, 2, 4, 1, 4, 5, 3, 3, 4, 3, 4, 1, 5, 2, 5, 2, 5, 5, 3, 3, 4, 3, 4, 2, 2, 2, 5, 2, 4, 4, 2, 2, 3, 2, 3, 5, 2, 2, 5, 3, 3, 4, 2, 4, 4, 4, 3, 4, 2, 3, 2, 3, 2, 2, 5, 2, 4, 2, 5, 5, 1, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 3, 2, 4, 2, 3, 2, 5, 2, 2, 4, 4, 4, 5, 2, 2, 3, 3, 2, 2, 4, 2, 4, 2, 4, 5, 2, 2, 5, 4, 2, 3, 2, 1, 5, 2, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 4, 3]

count = 0
for gpt, predict in zip(gpt, predictsdeep):
    if gpt == predict:
        count += 1
print(count)

In [ ]:
predictsdeep=[]
from collections import Counter
with open('GPT_with_new.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for item in lines:
        elements = item.split(': ')[1].strip()
        predictsdeep.append(int(elements))
print(predictsdeep)
print(len(predictsdeep))
save_predictions_to_jsonl(predictsdeep, 'predictions_gpt_new_rationale.jsonl')